# Adapter Pattern Tutorial 🔌

## What is the Adapter Pattern?

The Adapter Pattern allows objects with incompatible interfaces to collaborate. It acts as a wrapper between two objects, catching calls for one object and transforming them to format and interface recognizable by the second object.

**Real-world analogy**: Think of a power adapter when traveling abroad. Your laptop charger has a US plug, but the wall outlet in Europe has a different shape. The power adapter doesn't change the electricity or your charger - it just makes them compatible by translating the interface.

## Table of Contents
1. [What is the Adapter Pattern?](#what-is-the-adapter-pattern)
2. [Why Do We Need It?](#why-do-we-need-it)
3. [Simple Implementation](#simple-implementation)
4. [Understanding the Implementation](#understanding-the-implementation)
5. [Data Source Adapters (Advanced)](#advanced-example-data-source-adapters)
6. [When to Use (and When NOT to Use)](#when-to-use-and-when-not-to-use)

## Why Do We Need It?

Let's see what happens when we try to integrate incompatible systems:

In [ ]:
# Problem: Incompatible interfaces

# Our existing media player interface
class MediaPlayer:
    def play(self, audio_type: str, filename: str):
        if audio_type.lower() == "mp3":
            print(f"Playing MP3 file: {filename}")
        else:
            print(f"Unsupported audio format: {audio_type}")

# Third-party libraries with different interfaces
class AdvancedAudioPlayer:
    def play_mp4(self, filename: str):
        print(f"Playing MP4 audio: {filename}")
    
    def play_vlc(self, filename: str):
        print(f"Playing VLC media: {filename}")

class LegacyVideoPlayer:
    def play_avi_video(self, video_file: str):
        print(f"Playing AVI video: {video_file}")
    
    def play_mov_video(self, video_file: str):
        print(f"Playing MOV video: {video_file}")

# Problems:
# 1. Different method names (play vs play_mp4 vs play_avi_video)
# 2. Different parameter structures
# 3. Can't use these players directly with our MediaPlayer interface

# Current limitations
player = MediaPlayer()
player.play("mp3", "song.mp3")  # Works
player.play("mp4", "song.mp4")  # Doesn't work - unsupported format

# We can't easily integrate the advanced players
advanced_player = AdvancedAudioPlayer()
advanced_player.play_mp4("song.mp4")  # Works but incompatible interface

# This makes it hard to build a unified media system

## Simple Implementation

Let's solve this with the Adapter Pattern:

In [ ]:
from abc import ABC, abstractmethod

# Target interface (what our client expects)
class MediaPlayerInterface(ABC):
    @abstractmethod
    def play(self, audio_type: str, filename: str):
        pass

# Adaptee classes (third-party libraries with incompatible interfaces)
class AdvancedAudioPlayer:
    def play_mp4(self, filename: str):
        print(f"Playing MP4 audio: {filename}")
    
    def play_vlc(self, filename: str):
        print(f"Playing VLC media: {filename}")

class LegacyVideoPlayer:
    def play_avi_video(self, video_file: str):
        print(f"Playing AVI video: {video_file}")
    
    def play_mov_video(self, video_file: str):
        print(f"Playing MOV video: {video_file}")

# Adapter classes
class AdvancedAudioAdapter(MediaPlayerInterface):
    def __init__(self):
        self.advanced_player = AdvancedAudioPlayer()
    
    def play(self, audio_type: str, filename: str):
        if audio_type.lower() == "mp4":
            self.advanced_player.play_mp4(filename)
        elif audio_type.lower() == "vlc":
            self.advanced_player.play_vlc(filename)
        else:
            print(f"Unsupported format for AdvancedAudioPlayer: {audio_type}")

class VideoAdapter(MediaPlayerInterface):
    def __init__(self):
        self.video_player = LegacyVideoPlayer()
    
    def play(self, audio_type: str, filename: str):
        # Note: treating video formats as "audio_type" for consistency with interface
        if audio_type.lower() == "avi":
            self.video_player.play_avi_video(filename)
        elif audio_type.lower() == "mov":
            self.video_player.play_mov_video(filename)
        else:
            print(f"Unsupported format for VideoPlayer: {audio_type}")

# Enhanced MediaPlayer that uses adapters
class UniversalMediaPlayer(MediaPlayerInterface):
    def __init__(self):
        self.audio_adapter = AdvancedAudioAdapter()
        self.video_adapter = VideoAdapter()
    
    def play(self, audio_type: str, filename: str):
        # Handle native formats
        if audio_type.lower() == "mp3":
            print(f"Playing MP3 file: {filename}")
        
        # Delegate to adapters for other formats
        elif audio_type.lower() in ["mp4", "vlc"]:
            self.audio_adapter.play(audio_type, filename)
        
        elif audio_type.lower() in ["avi", "mov"]:
            self.video_adapter.play(audio_type, filename)
        
        else:
            print(f"Unsupported media format: {audio_type}")

# Testing the adapter pattern
print("--- Testing Universal Media Player with Adapters ---")

player = UniversalMediaPlayer()

# Native format
player.play("mp3", "song.mp3")

# Formats handled by AdvancedAudioAdapter
player.play("mp4", "movie_soundtrack.mp4")
player.play("vlc", "presentation.vlc")

# Formats handled by VideoAdapter
player.play("avi", "old_movie.avi")
player.play("mov", "quicktime_video.mov")

# Unsupported format
player.play("wav", "audio.wav")

## Understanding the Implementation

### Key Concepts:

1. **Target Interface**: The interface that the client expects (MediaPlayerInterface)
2. **Adaptee**: The existing class with incompatible interface (AdvancedAudioPlayer, LegacyVideoPlayer)
3. **Adapter**: The class that makes the adaptee compatible with the target interface
4. **Client**: The class that uses the target interface (UniversalMediaPlayer)

### Benefits:
- **Reuse**: Can reuse existing classes with incompatible interfaces
- **Separation of Concerns**: Interface adaptation is separate from business logic
- **Single Responsibility**: Each adapter handles one specific incompatibility
- **Open/Closed Principle**: Can add new adapters without modifying existing code

## Advanced Example: Data Source Adapters

Let's create a more complex example with different data sources:

In [ ]:
from abc import ABC, abstractmethod
from typing import List, Dict, Any
import json
import csv
from io import StringIO

# Target interface for data access
class DataSourceInterface(ABC):
    @abstractmethod
    def get_data(self) -> List[Dict[str, Any]]:
        """Return data as a list of dictionaries"""
        pass
    
    @abstractmethod
    def add_record(self, record: Dict[str, Any]) -> bool:
        """Add a new record"""
        pass

# Legacy database system with different interface
class LegacyDatabase:
    def __init__(self):
        # Simulating legacy database with different structure
        self.records = [
            "1|John Doe|john@example.com|25",
            "2|Jane Smith|jane@example.com|30",
            "3|Bob Johnson|bob@example.com|35"
        ]
    
    def fetch_all_records(self) -> List[str]:
        """Returns pipe-separated strings"""
        return self.records.copy()
    
    def insert_record(self, pipe_separated_record: str) -> int:
        """Insert record and return ID"""
        record_id = len(self.records) + 1
        self.records.append(pipe_separated_record)
        return record_id

# External API system
class ExternalAPIClient:
    def __init__(self):
        # Simulating API responses
        self.api_data = {
            "users": [
                {"user_id": 101, "full_name": "Alice Wilson", "email_address": "alice@api.com", "years_old": 28},
                {"user_id": 102, "full_name": "Charlie Brown", "email_address": "charlie@api.com", "years_old": 22}
            ],
            "status": "success"
        }
    
    def get_users(self) -> Dict[str, Any]:
        """Returns API response with nested structure"""
        return self.api_data
    
    def create_user(self, user_data: Dict[str, Any]) -> Dict[str, Any]:
        """Create user via API"""
        new_id = max([u["user_id"] for u in self.api_data["users"]]) + 1
        new_user = {
            "user_id": new_id,
            "full_name": user_data["full_name"],
            "email_address": user_data["email_address"],
            "years_old": user_data["years_old"]
        }
        self.api_data["users"].append(new_user)
        return {"status": "created", "user_id": new_id}

# CSV file system
class CSVFileHandler:
    def __init__(self):
        # Simulating CSV content
        self.csv_content = """id,name,email,age
201,"Diana Prince",diana@csv.com,29
202,"Clark Kent",clark@csv.com,32
203,"Bruce Wayne",bruce@csv.com,35"""
    
    def read_csv(self) -> str:
        return self.csv_content
    
    def append_to_csv(self, row_data: List[str]) -> bool:
        new_row = ",".join([f'"{item}"' for item in row_data])
        self.csv_content += "\n" + new_row
        return True

# Adapter for Legacy Database
class LegacyDatabaseAdapter(DataSourceInterface):
    def __init__(self, legacy_db: LegacyDatabase):
        self.legacy_db = legacy_db
    
    def get_data(self) -> List[Dict[str, Any]]:
        records = self.legacy_db.fetch_all_records()
        result = []
        
        for record in records:
            parts = record.split('|')
            if len(parts) == 4:
                result.append({
                    "id": int(parts[0]),
                    "name": parts[1],
                    "email": parts[2],
                    "age": int(parts[3])
                })
        
        return result
    
    def add_record(self, record: Dict[str, Any]) -> bool:
        pipe_record = f"{record['id']}|{record['name']}|{record['email']}|{record['age']}"
        record_id = self.legacy_db.insert_record(pipe_record)
        return record_id > 0

# Adapter for External API
class ExternalAPIAdapter(DataSourceInterface):
    def __init__(self, api_client: ExternalAPIClient):
        self.api_client = api_client
    
    def get_data(self) -> List[Dict[str, Any]]:
        response = self.api_client.get_users()
        
        if response["status"] == "success":
            result = []
            for user in response["users"]:
                result.append({
                    "id": user["user_id"],
                    "name": user["full_name"],
                    "email": user["email_address"],
                    "age": user["years_old"]
                })
            return result
        
        return []
    
    def add_record(self, record: Dict[str, Any]) -> bool:
        api_record = {
            "full_name": record["name"],
            "email_address": record["email"],
            "years_old": record["age"]
        }
        
        response = self.api_client.create_user(api_record)
        return response.get("status") == "created"

# Adapter for CSV File
class CSVFileAdapter(DataSourceInterface):
    def __init__(self, csv_handler: CSVFileHandler):
        self.csv_handler = csv_handler
    
    def get_data(self) -> List[Dict[str, Any]]:
        csv_content = self.csv_handler.read_csv()
        csv_reader = csv.DictReader(StringIO(csv_content))
        
        result = []
        for row in csv_reader:
            result.append({
                "id": int(row["id"]),
                "name": row["name"],
                "email": row["email"],
                "age": int(row["age"])
            })
        
        return result
    
    def add_record(self, record: Dict[str, Any]) -> bool:
        row_data = [str(record["id"]), record["name"], record["email"], str(record["age"])]
        return self.csv_handler.append_to_csv(row_data)

# Universal data manager that works with any data source
class DataManager:
    def __init__(self):
        self.data_sources: List[DataSourceInterface] = []
    
    def add_data_source(self, data_source: DataSourceInterface):
        self.data_sources.append(data_source)
    
    def get_all_data(self) -> List[Dict[str, Any]]:
        all_data = []
        for source in self.data_sources:
            data = source.get_data()
            all_data.extend(data)
        return all_data
    
    def add_record_to_source(self, source_index: int, record: Dict[str, Any]) -> bool:
        if 0 <= source_index < len(self.data_sources):
            return self.data_sources[source_index].add_record(record)
        return False
    
    def get_summary(self) -> Dict[str, Any]:
        all_data = self.get_all_data()
        return {
            "total_records": len(all_data),
            "average_age": sum(record["age"] for record in all_data) / len(all_data) if all_data else 0,
            "data_sources": len(self.data_sources)
        }

# Testing the data source adapters
print("--- Testing Data Source Adapters ---")

# Create data sources
legacy_db = LegacyDatabase()
api_client = ExternalAPIClient()
csv_handler = CSVFileHandler()

# Create adapters
legacy_adapter = LegacyDatabaseAdapter(legacy_db)
api_adapter = ExternalAPIAdapter(api_client)
csv_adapter = CSVFileAdapter(csv_handler)

# Create data manager and add sources
data_manager = DataManager()
data_manager.add_data_source(legacy_adapter)
data_manager.add_data_source(api_adapter)
data_manager.add_data_source(csv_adapter)

# Get data from all sources using uniform interface
print("All data from different sources:")
all_data = data_manager.get_all_data()
for record in all_data:
    print(f"  ID: {record['id']}, Name: {record['name']}, Email: {record['email']}, Age: {record['age']}")

print(f"\nSummary: {data_manager.get_summary()}")

# Add new records to different sources
print("\nAdding new records...")
new_record = {"id": 999, "name": "New User", "email": "new@example.com", "age": 25}

# Add to legacy database (index 0)
success = data_manager.add_record_to_source(0, new_record)
print(f"Added to legacy database: {success}")

# Verify the record was added
updated_data = legacy_adapter.get_data()
print(f"Legacy database now has {len(updated_data)} records")

## Object Adapter vs Class Adapter

There are two main types of adapter patterns. Let's demonstrate both:

In [ ]:
# Target interface
class CalculatorInterface(ABC):
    @abstractmethod
    def calculate(self, a: float, b: float, operation: str) -> float:
        pass

# Adaptee - scientific calculator with different interface
class ScientificCalculator:
    def add(self, x: float, y: float) -> float:
        return x + y
    
    def subtract(self, x: float, y: float) -> float:
        return x - y
    
    def multiply(self, x: float, y: float) -> float:
        return x * y
    
    def divide(self, x: float, y: float) -> float:
        if y == 0:
            raise ValueError("Division by zero")
        return x / y
    
    def power(self, base: float, exponent: float) -> float:
        return base ** exponent

# Object Adapter (Composition) - more flexible
class CalculatorObjectAdapter(CalculatorInterface):
    def __init__(self, scientific_calc: ScientificCalculator):
        self.scientific_calc = scientific_calc
    
    def calculate(self, a: float, b: float, operation: str) -> float:
        operation = operation.lower()
        
        if operation == "add" or operation == "+":
            return self.scientific_calc.add(a, b)
        elif operation == "subtract" or operation == "-":
            return self.scientific_calc.subtract(a, b)
        elif operation == "multiply" or operation == "*":
            return self.scientific_calc.multiply(a, b)
        elif operation == "divide" or operation == "/":
            return self.scientific_calc.divide(a, b)
        elif operation == "power" or operation == "**":
            return self.scientific_calc.power(a, b)
        else:
            raise ValueError(f"Unsupported operation: {operation}")

# Class Adapter (Inheritance) - less flexible but can override methods
class CalculatorClassAdapter(ScientificCalculator, CalculatorInterface):
    def calculate(self, a: float, b: float, operation: str) -> float:
        operation = operation.lower()
        
        if operation == "add" or operation == "+":
            return self.add(a, b)
        elif operation == "subtract" or operation == "-":
            return self.subtract(a, b)
        elif operation == "multiply" or operation == "*":
            return self.multiply(a, b)
        elif operation == "divide" or operation == "/":
            return self.divide(a, b)
        elif operation == "power" or operation == "**":
            return self.power(a, b)
        else:
            raise ValueError(f"Unsupported operation: {operation}")
    
    # Can override adaptee methods if needed
    def divide(self, x: float, y: float) -> float:
        if y == 0:
            print("Warning: Division by zero, returning infinity")
            return float('inf')
        return super().divide(x, y)

# Testing both adapter types
print("--- Testing Object vs Class Adapters ---")

# Object Adapter
scientific_calc = ScientificCalculator()
object_adapter = CalculatorObjectAdapter(scientific_calc)

print("Object Adapter:")
print(f"10 + 5 = {object_adapter.calculate(10, 5, 'add')}")
print(f"10 - 5 = {object_adapter.calculate(10, 5, 'subtract')}")
print(f"10 * 5 = {object_adapter.calculate(10, 5, 'multiply')}")
print(f"10 / 5 = {object_adapter.calculate(10, 5, 'divide')}")
print(f"10 ** 2 = {object_adapter.calculate(10, 2, 'power')}")

# Class Adapter
class_adapter = CalculatorClassAdapter()

print("\nClass Adapter:")
print(f"10 + 5 = {class_adapter.calculate(10, 5, 'add')}")
print(f"10 - 5 = {class_adapter.calculate(10, 5, 'subtract')}")
print(f"10 * 5 = {class_adapter.calculate(10, 5, 'multiply')}")
print(f"10 / 5 = {class_adapter.calculate(10, 5, 'divide')}")
print(f"10 / 0 = {class_adapter.calculate(10, 0, 'divide')}")  # Shows overridden behavior

# Can also access adaptee methods directly in class adapter
print(f"\nDirect access in class adapter: {class_adapter.add(7, 3)}")

## Two-way Adapter

Sometimes you need an adapter that can work in both directions:

In [ ]:
# Two interfaces that need to work together
class TemperatureCelsius:
    def __init__(self, temperature: float):
        self.celsius = temperature
    
    def get_celsius(self) -> float:
        return self.celsius
    
    def set_celsius(self, temp: float):
        self.celsius = temp
    
    def __str__(self):
        return f"{self.celsius}°C"

class TemperatureFahrenheit:
    def __init__(self, temperature: float):
        self.fahrenheit = temperature
    
    def get_fahrenheit(self) -> float:
        return self.fahrenheit
    
    def set_fahrenheit(self, temp: float):
        self.fahrenheit = temp
    
    def __str__(self):
        return f"{self.fahrenheit}°F"

# Two-way adapter that implements both interfaces
class TemperatureTwoWayAdapter:
    def __init__(self, celsius_temp: TemperatureCelsius = None, fahrenheit_temp: TemperatureFahrenheit = None):
        if celsius_temp:
            self._celsius = celsius_temp.get_celsius()
        elif fahrenheit_temp:
            self._celsius = self._fahrenheit_to_celsius(fahrenheit_temp.get_fahrenheit())
        else:
            self._celsius = 0.0
    
    # Celsius interface methods
    def get_celsius(self) -> float:
        return self._celsius
    
    def set_celsius(self, temp: float):
        self._celsius = temp
    
    # Fahrenheit interface methods
    def get_fahrenheit(self) -> float:
        return self._celsius_to_fahrenheit(self._celsius)
    
    def set_fahrenheit(self, temp: float):
        self._celsius = self._fahrenheit_to_celsius(temp)
    
    # Conversion methods
    def _celsius_to_fahrenheit(self, celsius: float) -> float:
        return (celsius * 9/5) + 32
    
    def _fahrenheit_to_celsius(self, fahrenheit: float) -> float:
        return (fahrenheit - 32) * 5/9
    
    def __str__(self):
        return f"{self._celsius}°C / {self.get_fahrenheit()}°F"

# Functions that work with specific temperature types
def celsius_weather_report(temp: TemperatureCelsius):
    celsius = temp.get_celsius()
    if celsius < 0:
        return f"Freezing cold at {temp}"
    elif celsius < 10:
        return f"Cold at {temp}"
    elif celsius < 25:
        return f"Mild at {temp}"
    else:
        return f"Warm at {temp}"

def fahrenheit_weather_report(temp: TemperatureFahrenheit):
    fahrenheit = temp.get_fahrenheit()
    if fahrenheit < 32:
        return f"Below freezing at {temp}"
    elif fahrenheit < 50:
        return f"Cold at {temp}"
    elif fahrenheit < 77:
        return f"Pleasant at {temp}"
    else:
        return f"Hot at {temp}"

# Testing two-way adapter
print("--- Testing Two-Way Temperature Adapter ---")

# Create adapter from Celsius
celsius_temp = TemperatureCelsius(25.0)
adapter = TemperatureTwoWayAdapter(celsius_temp=celsius_temp)

print(f"Original: {celsius_temp}")
print(f"Adapter: {adapter}")

# Use adapter with Celsius function
print(f"Celsius report: {celsius_weather_report(adapter)}")

# Use adapter with Fahrenheit function
print(f"Fahrenheit report: {fahrenheit_weather_report(adapter)}")

# Change temperature through Fahrenheit interface
adapter.set_fahrenheit(32.0)  # Freezing point
print(f"\nAfter setting to 32°F: {adapter}")
print(f"Celsius report: {celsius_weather_report(adapter)}")
print(f"Fahrenheit report: {fahrenheit_weather_report(adapter)}")

# Change temperature through Celsius interface
adapter.set_celsius(30.0)  # Hot day
print(f"\nAfter setting to 30°C: {adapter}")
print(f"Celsius report: {celsius_weather_report(adapter)}")
print(f"Fahrenheit report: {fahrenheit_weather_report(adapter)}")

## When to Use (and When NOT to Use)

### Use Adapter Pattern when:
- You want to use an existing class with an incompatible interface
- You need to integrate third-party libraries with different interfaces
- You want to create a reusable class that cooperates with unrelated classes
- You need to use several existing subclasses but it's impractical to adapt their interface by subclassing
- You're working with legacy code that you can't modify

### Don't use when:
- The interfaces are already compatible
- You can easily modify the existing classes
- The adaptation logic is too complex and would be better handled differently
- Performance overhead of the adapter layer is significant

### Real-world applications:
- Database drivers (JDBC, ODBC)
- Legacy system integration
- Third-party library integration
- File format converters
- Payment gateway integration
- API versioning (v1 to v2 compatibility)
- Media format adapters (like our first example)
- Cross-platform compatibility layers
- ORM frameworks adapting different database interfaces